In [9]:
from typing import Type
from pydantic import BaseModel, Field

PORT_NUM = 5000
SERVER_URL = "http://localhost:" + str(PORT_NUM)

def generate_field_description(model: Type[BaseModel]) -> str:
    """
    Takes a Pydantic model class and generates a formatted field description.
    """
    field_descriptions = []
    
    # Loop through each field in the Pydantic model
    for field_name, field_info in model.model_fields.items():
        field_type = field_info.annotation.__name__  # Get the type name
        field_desc = field_info.description if field_info.description else ""
        
        # Format field description
        field_descriptions.append(f"- {field_name} [{field_type}]: {field_desc}")

    # Build the JSON format string correctly (avoid f-string issues)
    json_example = "{\n  " + ",\n  ".join(
        [f'"{name}": <{model.model_fields[name].annotation.__name__}>' 
         for name in model.model_fields]
    ) + "\n}"

    # Construct the final output
    json_output = (
        "Generate a JSON response. The fields should include: \n" + 
        "\n".join(field_descriptions) +
        "\n\nPlease provide the next response **only** in the following valid JSON format, without any additional text:\n\n" +
        json_example
    )
    
    return json_output

In [10]:
import requests

# Define the server URL (update if needed)
HEALTH_URL = SERVER_URL + "/health"

try:
    # Send a GET request
    response = requests.get(HEALTH_URL)

    # Check if request succeeded (HTTP status code 200)
    if response.status_code == 200:
        print(f"Success: Server responded with {response.status_code}")
        print("Response Content:", response.text)
    else:
        print(f"Warning: Server responded with status code {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Error: Could not connect to the server.\nDetails: {e}")

Success: Server responded with 200
Response Content: {"status":"running"}



In [11]:
class User(BaseModel):
    name: str = Field(...,description="The user's name")
    age: int = Field(...,description="The user's age")
    hobbies: list[str] = Field(..., description="A list of the user's hobbies.")

In [12]:
print(generate_field_description(User))

Generate a JSON response. The fields should include: 
- name [str]: The user's name
- age [int]: The user's age
- hobbies [list]: A list of the user's hobbies.

Please provide the next response **only** in the following valid JSON format, without any additional text:

{
  "name": <str>,
  "age": <int>,
  "hobbies": <list>
}


In [13]:
User.model_fields
# User.__pydantic_fields__

{'name': FieldInfo(annotation=str, required=True, description="The user's name"),
 'age': FieldInfo(annotation=int, required=True, description="The user's age"),
 'hobbies': FieldInfo(annotation=list[str], required=True, description="A list of the user's hobbies.")}

In [14]:
import requests
import json


CHAT_URL = SERVER_URL + "/chat"

# Define the request payload (matches the curl command)
payload = {
    "model": "llama3.3:latest",
    "stream": False,
    "messages": [
        {
            "role": "system",
            "content": (
              generate_field_description(User)
            ),
        },
        {
            "role": "user",
            "content": "hi my name is nancy, I'm 25, and I like walking my dog. I also like eating avocados and sleeping."
        }
    ]
}

# Set the headers for JSON request
headers = {"Content-Type": "application/json"}

try:
    # Send POST request to the Flask API
    response = requests.post(CHAT_URL, headers=headers, json=payload)

    # Check if the request was successful
    if response.status_code == 200:
        response_json = response.json()
        raw_content = response_json["message"]["content"]
        print(raw_content)

    else:
        print(f"Error {response.status_code}: {response.text}")

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

{
  "name": "Nancy",
  "age": 25,
  "hobbies": ["walking my dog", "eating avocados", "sleeping"]
}
